In [167]:
from selenium import webdriver
from selenium.webdriver.common.by import By # TAG에 접근하기 편하게 하는 기능
from selenium.webdriver.common.keys import Keys # 키보드 이벤트
import pandas as pd
from sqlalchemy import create_engine

In [168]:
# 웹드라이버를 이용하여 구글크롬 브라우저를 오픈
driver = webdriver.Chrome()

In [169]:
# 특정 주소로 요청을 보낸다.
driver.get('http://www.naver.com')

In [170]:
# Tag 중 id가 query인 태그를 선택하여 변수에 저장
search_element = driver.find_element(By.ID, 'query')

In [171]:
search_element

<selenium.webdriver.remote.webelement.WebElement (session="3ec6cbb8ff6f9ae578d7a99086ea690b", element="f.4660CA44801EA1A4DCA3C655296ADF64.d.1BC4E166A4C8C3E5AAA8F4F3C85ED9D9.e.4")>

In [172]:
# search_element(검색어창)에 특정 텍스트를 입력(데이터를 보낸다)한다.
search_element.send_keys('아이폰')

In [173]:
# search_element에서 ENTER 이벤트를 발생
search_element.send_keys(Keys.ENTER)

In [174]:
# 쇼핑 버튼(Tag)을 선택한다.
# LINK_TEXT가 '쇼핑'인 태그 -> 해당 조건에 맞는 태그가 몆개인가? -> len, find_elements 사용 
# -> 결과 1개: 유일함 사용가능
len(driver.find_elements(By.LINK_TEXT, '쇼핑'))
# 조건의 맞는 태그의 개수가 1이므로 클릭 이벤트 발생
driver.find_element(By.LINK_TEXT, '쇼핑').click()

In [175]:
# 쇼핑 페이지에서 검색어의 결과(물건의 가격과 제품명)를 크롤링 -> 해당 페이지의 소스코드를 불러온다.
html_data = driver.page_source


In [176]:
# bs4 라이브러리 안에 있는 BeautifulSoup을 이용하여 html_data를 parsing(데이터의 타입을 변환)
from bs4 import BeautifulSoup as bs

In [177]:
soup = bs(html_data, 'html.parser')


In [178]:
# html_data가 2개의 탭중 어떤 소스코드인지 확인
# 탭의 제목이 다르기 때문에 title Tag 확인
# 특정 태그를 확인
soup.title


<title>아이폰 : 네이버 검색</title>

In [179]:
# page_source가 첫번째 탭인것을 확인
# 두번째 탭으로 page_source를 이동
# 탬의 정보를 확인
# driver에서 탭을 주소를 확인
driver.window_handles

['4660CA44801EA1A4DCA3C655296ADF64', '56FDF624F813FA2A0E2BCBFD752361F9']

In [180]:
# driver에서 탭을 이동
driver.switch_to.window(
    driver.window_handles[1]
)

In [181]:
html_data2 = driver.page_source
soup2 = bs(html_data2, 'html.parser')

soup.title

<title>아이폰 : 네이버 검색</title>

#### 네이버 쇼핑 크롤링
1. div 태그중 id가 'content'인 태그를 선택하여 저장 (content_data)
2. 저장한 태그(content_data)에서 div 태그중 class가 'product_item__KQayS'인 태그 정보를 모두 저장
3. div_list에서 div 태그중 class가 'product_title_1jFM_'인 태그의 문자를 추출(상품명)
4. div_class에서 span 태그중 class가 'price'인 태그의 문자를 추출(상품 격)

In [182]:
content_data = soup2.find(
    'div',
    attrs = {
        'id' : 'content'
    }
)

In [183]:
import re

In [184]:
# content_data에서 div중 class가 'product_item__KQayS'인 태그의 개수를 확인
# class의 이름에 'product_item'이 포함되어있는 태그
len (
    content_data.find_all(
        'div',
        # attrs = {
        #     'class' : 'product_item__KQayS'
        #     }
        attrs = {
            'class' : re.compile('product_item')
        }
    )
)


3

In [185]:
div_list = content_data.find_all(
    'div',
    attrs = {
        'class' : 'product_item__KQayS'
    }
)

In [186]:
div_data = div_list[0]
div_data

<div class="product_item__KQayS"><div class="product_inner__O9F_g"><div class="product_img_area__ziVZA"><div class="thumbnail_thumb_wrap__X013_ _wrapper"><a class="thumbnail_thumb__MG0r2 linkAnchor _nlog_click _nlog_impression_element" data-shp-area="lst*N.img" data-shp-area-dtl='[{"key":"trtr","value":"slsl"}]' data-shp-area-id="img" data-shp-area-type="slot" data-shp-contents-dtl='[{"key":"prod_nm","value":"Apple 애플 아이폰 13 미니 128GB 새상품"},{"key":"price","value":"298000"},{"key":"brand_seq","value":"144225"},{"key":"nfa_type","value":"NORMAL"},{"key":"catalog_nv_mid","value":"88573763568"},{"key":"organic_expose_order","value":"1"},{"key":"chnl_prod_no","value":"11029257361"},{"key":"std_yn","value":"n"}]' data-shp-contents-grp="prod" data-shp-contents-id="88573763568" data-shp-contents-provider-dtl='[{"key":"adsr_type","value":"SHOPN"},{"key":"brandstore_type","value":"n"},{"key":"chnl_no","value":"102459197"}]' data-shp-contents-provider-id="11501384" data-shp-contents-provider-type=

In [187]:
item_name =  div_data.find(
    'div',
    # attrs = {
    #     'class' : 'product_title__ljFM_'
    # }
    attrs = {
        'class' : re.compile('product_title')
    }
).get_text()

In [188]:
item_price = div_data.find(
    'span', 
    attrs = { 'class' : 'price'}
    ).get_text()

In [189]:
# 상품의 이름과 상품의 가격을 dict 형태로 생성

# 비어있는 dict 생성
dict_data = {}

In [190]:
# dict에 데이터를 추가
dict_data['상품명'] = item_name
dict_data['가격'] = item_price

In [191]:
dict_data

{'상품명': 'Apple 애플 아이폰 13 미니 128GB 새상품', '가격': '298,000원'}

In [192]:
values = []
for div_data in div_list:
    item_name =  div_data.find(
        'div',
        # attrs = {
        #     'class' : 'product_title__ljFM_'
        # }
        attrs = {
            'class' : re.compile('product_title')
        }
    ).get_text()
    item_price = div_data.find(
        'span', 
        attrs = { 'class' : 'price'}
    ).get_text()
    item_url = div_data.find('a')['href']
    dict_data = {
        '상품명' : item_name,
        '가격' : item_price,
        'url' : item_url
    }
    values.append(dict_data)

In [193]:
values

[{'상품명': 'Apple 애플 아이폰 13 미니 128GB 새상품',
  '가격': '298,000원',
  'url': 'https://cr.shopping.naver.com/adcr?x=Ck%2FvsrXmwBwnevbM4QrIoP%2F%2F%2Fw%3D%3Ds8tkMvDQe32FPfURcsI0ElVmkv240GU1MzK0XLQcDkyHxTAJR5s3V3RheLYZrm3hTYpy37YyyU%2ByFcekuVr%2FDKmg6HabDTRx466CGciRiUnFBZuMfzSBinsMMtj7lqMfU1m01%2BgVyS3AXFrpnOyT1J5ztC8eyDbGRDg%2BBSXyJFwhIJAugUYaa7eEC0K1redZoRL6iu0rN9ANYEXkZX8p%2BoSEm%2FeWMmjv1VUimPr%2BbdY5YMzOyJ4Aj7ODRdJa%2BvbUnQ1pu9SxBx9jmyq0tgSOnxCGHDvsGQoUHd%2FG2Ez5s1IA%2Fa18qhDQ1A%2FKJp3fgBDor391W%2FC1w5SVs2sYtTdFSejRuJqqFub1Vv4MTzhrNk2Ehy7i1mLAodI8Q%2BJ0drSjw%2F0X8gFBBhr6rnnTk5Nq2rPVbEXVgpkByUYUZnAOmWLrzH6N5zxqs3SNW2xsM3EczzWEAAvAfCXV8J8EBeiSYUAPzQvOQpktRn2nqaAl90BjPyDwmo6hkNyPBbWcewE89C%2Bp633m53JCQqoKkv9VJyLB61z4X6oeURmaMgKrJTZajrgWQ%2F0JBsjHud4pzo5y1ej6SapLSoWv5geXOQmBRuSLOIjWF4Q%2B20M82%2B8R5In0%2FFJIMHKNa%2FriGGHnq4ZPB5Hv1T%2FoDPeAT2ZD%2BWCfnsQ%3D%3D&nvMid=88573763568&catId=50001519'},
 {'상품명': '아이폰 16 프로 128GB [자급제]',
  '가격': '최저1,538,900원',
  'url': 'https://cr.shopping.naver.com/adcr

In [194]:
# div_list에서 div 태그 중 class의 이름에 'product_img_area'가 포함된 태그를 찾는다.
# 해당 태그에서 img 태그에 있는 src 속성의 값을 추출
div_data = div_list[0]

In [195]:
# div_data에서 a태그(하이퍼링크) 첫번째 정보를 확인하여 href 속성의 값을 출력
# html문서에서 첫 번째 태그의 정보를 확인
    # htmldata.태그명
    # htmldata.find(태그명)
# 특정 태그의 속성의 값을 확인
    # Tag[속성명]
a_tag = div_data.find('a')
a_tag['href']

'https://cr.shopping.naver.com/adcr?x=Ck%2FvsrXmwBwnevbM4QrIoP%2F%2F%2Fw%3D%3Ds8tkMvDQe32FPfURcsI0ElVmkv240GU1MzK0XLQcDkyHxTAJR5s3V3RheLYZrm3hTYpy37YyyU%2ByFcekuVr%2FDKmg6HabDTRx466CGciRiUnFBZuMfzSBinsMMtj7lqMfU1m01%2BgVyS3AXFrpnOyT1J5ztC8eyDbGRDg%2BBSXyJFwhIJAugUYaa7eEC0K1redZoRL6iu0rN9ANYEXkZX8p%2BoSEm%2FeWMmjv1VUimPr%2BbdY5YMzOyJ4Aj7ODRdJa%2BvbUnQ1pu9SxBx9jmyq0tgSOnxCGHDvsGQoUHd%2FG2Ez5s1IA%2Fa18qhDQ1A%2FKJp3fgBDor391W%2FC1w5SVs2sYtTdFSejRuJqqFub1Vv4MTzhrNk2Ehy7i1mLAodI8Q%2BJ0drSjw%2F0X8gFBBhr6rnnTk5Nq2rPVbEXVgpkByUYUZnAOmWLrzH6N5zxqs3SNW2xsM3EczzWEAAvAfCXV8J8EBeiSYUAPzQvOQpktRn2nqaAl90BjPyDwmo6hkNyPBbWcewE89C%2Bp633m53JCQqoKkv9VJyLB61z4X6oeURmaMgKrJTZajrgWQ%2F0JBsjHud4pzo5y1ej6SapLSoWv5geXOQmBRuSLOIjWF4Q%2B20M82%2B8R5In0%2FFJIMHKNa%2FriGGHnq4ZPB5Hv1T%2FoDPeAT2ZD%2BWCfnsQ%3D%3D&nvMid=88573763568&catId=50001519'

In [196]:
# driver 스크룰을 가장 밑으로 내린다.
# driver.execute_script(
#     "window.scrollTo(0, document.body.scrollHeight);"
# )

In [202]:
# driver에서 스크롤을 일정 간격으로 내린다.
driver.execute_script(
    "window.scrollBy(0,800);"
)

In [215]:
# BeautifulSoup을 이용해서 데이터 파싱
html_data3 = driver.page_source
soup3 = bs(html_data3, 'html.parser')
content_data = soup3.find(
    'div',
    attrs = {
        'id' : 'content'
    }
)

div_list = content_data.find_all(
    'div',
    attrs = {
        'class' : 'product_item__KQayS'
    }
)
# 광고 상품을 제외한 모든 상품의 이름과 가격 링크 주소를 2차원 데이터(리스트 안에 딕셔너리)로 생성
values = []
for div_data in div_list:
    item_name =  div_data.find(
        'div',
        # attrs = {
        #     'class' : 'product_title__ljFM_'
        # }
        attrs = {
            'class' : re.compile('product_title')
        }
    ).get_text()
    item_price = div_data.find(
        'span', 
        attrs = { 'class' : 'price'}
    ).get_text()
    item_url = div_data.find('a')['href']
    dict_data = {
        '상품명' : item_name,
        '가격' : item_price,
        'url' : item_url
    }
    values.append(dict_data)

#생상된 2차원 데이터를 데이터프레임으로 생성
df = pd.DataFrame(values)

#검색어('아이폰')를 파일명으로 csv파일을 생성하고 인덱스는 제외
df.to_csv("아이폰.csv", index=False)


In [213]:
df

,상품명,가격,url
0,Apple 애플 아이폰 13 미니 128GB 새상품,"298,000원",https://cr.shopping.naver.com/adcr?x=Ck%2FvsrX...
1,아이폰 16 프로 128GB [자급제],"최저1,538,900원",https://cr.shopping.naver.com/adcr?x=Deao6k1eK...
2,아이폰 16 프로 256GB [자급제],"최저1,647,990원",https://cr.shopping.naver.com/adcr?x=8szemkTuY...
3,애플 아이폰 13 미니 128GB 미개봉 새제품,"298,000원",https://cr.shopping.naver.com/adcr?x=LF%2FG%2B...
4,아이폰 16 128GB [자급제],"최저1,150,000원",https://cr.shopping.naver.com/adcr?x=ksQdXpBiX...
5,아이폰 15 128GB [자급제],"최저949,900원",https://cr.shopping.naver.com/adcr?x=I98g6Wao6...
6,아이폰 16 256GB [자급제],"최저1,394,000원",https://cr.shopping.naver.com/adcr?x=DK4YYgSyQ...
7,아이폰 15 프로 256GB [자급제],"최저1,393,900원",https://cr.shopping.naver.com/adcr?x=vujnB65CR...
8,Apple 아이폰 12 mini 자급제 화이트 128GB,"458,000원",https://cr.shopping.naver.com/adcr?x=7GB%2BtDN...
9,해외아이폰 5S 무음 공기계 인스타 카메라 감성사진 학생폰 세컨폰 실버 16GB 항공,"79,500원",https://cr.shopping.naver.com/adcr?x=AYX%2BRoD...


In [217]:
# driver를 종료
driver.quit()